In [3]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, roc_curve, auc
from sklearn.datasets import make_classification
%config IPCompleter.greedy=True
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
sns.set_style("whitegrid")
sns.set(rc={'figure.figsize': (12, 6)})

In [4]:
data = pd.read_csv("../files/datasets_32132_205932_beauty.csv")

In [5]:
data.head()

,wage,exper,union,goodhlth,black,female,married,service,educ,looks
0,5.73,30,0,1,0,1,1,1,14,4
1,4.28,28,0,1,0,1,1,0,12,3
2,7.96,35,0,1,0,1,0,0,10,4
3,11.57,38,0,1,0,0,1,1,16,3
4,11.42,27,0,1,0,0,1,0,16,3


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1260 entries, 0 to 1259
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   wage      1260 non-null   float64
 1   exper     1260 non-null   int64  
 2   union     1260 non-null   int64  
 3   goodhlth  1260 non-null   int64  
 4   black     1260 non-null   int64  
 5   female    1260 non-null   int64  
 6   married   1260 non-null   int64  
 7   service   1260 non-null   int64  
 8   educ      1260 non-null   int64  
 9   looks     1260 non-null   int64  
dtypes: float64(1), int64(9)
memory usage: 98.6 KB


In [7]:
data.describe()

,wage,exper,union,goodhlth,black,female,married,service,educ,looks
count,1260.000000,1260.000000,1260.000000,1260.000000,1260.000000,1260.000000,1260.000000,1260.000000,1260.000000,1260.000000
mean,6.306690,18.206349,0.272222,0.933333,0.073810,0.346032,0.691270,0.273810,12.563492,3.185714
std,4.660639,11.963485,0.445280,0.249543,0.261564,0.475892,0.462153,0.446089,2.624489,0.684877
min,1.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,1.000000
25%,3.707500,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,12.000000,3.000000
50%,5.300000,15.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,12.000000,3.000000
75%,7.695000,27.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,13.000000,4.000000
max,77.720000,48.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,17.000000,5.000000


In [8]:
data[data["female"] == 1]["wage"].mean(), \
data[data["female"] == 0]["wage"].mean()

(4.299357798165137, 7.3688228155339806)

In [9]:
data[(data["female"] == 0) & (data["married"] == 1)]["wage"].median(), \
data[(data["female"] == 0) & (data["married"] == 0)]["wage"].median()

(6.710000000000001, 5.0649999999999995)

In [10]:
for look, sub_df in data.groupby("looks"):
    print(look)
    print(sub_df["female"].mean())

1
0.38461538461538464
2
0.38028169014084506
3
0.32271468144044324
4
0.37362637362637363
5
0.42105263157894735


In [12]:
data.groupby("looks")[["wage", "exper"]].agg(np.median)

,wage,exper
looks,,
1,3.460,32.0
2,4.595,18.0
3,5.635,18.0
4,5.240,12.5
5,4.810,8.0


In [83]:
pd.crosstab(data.female, data.looks, normalize=True)  # normalize - percents, margin - ALL

looks,1,2,3,4,5
female,,,,,
0,0.006349,0.069841,0.388095,0.180952,0.008730
1,0.003968,0.042857,0.184921,0.107937,0.006349


In [84]:
data["is_reach"] = (data["wage"] > 
                    data["wage"].quantile(.75)).astype("int64")

In [85]:
data.head()

,wage,exper,union,goodhlth,black,female,married,service,educ,looks,is_reach
0,5.73,30,0,1,0,1,1,1,14,4,0
1,4.28,28,0,1,0,1,1,0,12,3,0
2,7.96,35,0,1,0,1,0,0,10,4,1
3,11.57,38,0,1,0,0,1,1,16,3,1
4,11.42,27,0,1,0,0,1,0,16,3,1


In [86]:
data["female"].apply(lambda x: "female" if x else "male")

0       female
1       female
2       female
3         male
4         male
         ...  
1255    female
1256    female
1257    female
1258    female
1259    female
Name: female, Length: 1260, dtype: object

In [87]:
d = {0: "non-union", 1: "union"}

In [88]:
data["union"].map(d).head()

0    non-union
1    non-union
2    non-union
3    non-union
4    non-union
Name: union, dtype: object

In [13]:
################################################ dop experiments #########################################
data = pd.read_csv("../files/adult.data.csv")

In [14]:
data.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [16]:
data.groupby("education")[["age", "hours-per-week"]].describe(percentiles=[])

age                                         hours-per-week  \
                count       mean        std   min   50%   max          count   
education                                                                      
10th            933.0  37.429796  16.720713  17.0  34.0  90.0          933.0   
11th           1175.0  32.355745  15.545485  17.0  28.0  90.0         1175.0   
12th            433.0  32.000000  14.334625  17.0  28.0  79.0          433.0   
1st-4th         168.0  46.142857  15.615625  19.0  46.0  90.0          168.0   
5th-6th         333.0  42.885886  15.557285  17.0  42.0  84.0          333.0   
7th-8th         646.0  48.445820  16.092350  17.0  50.0  90.0          646.0   
9th             514.0  41.060311  15.946862  17.0  39.0  90.0          514.0   
Assoc-acdm     1067.0  37.381443  11.095177  19.0  36.0  90.0         1067.0   
Assoc-voc      1382.0  38.553546  11.631300  19.0  37.0  84.0         1382.0   
Bachelors      5355.0  38.904949  11.912210  19.0  37.0  90.0         5355.0   
Doctorate       413.0  47.702179  11.784716  24.0  47.0  80.0          413.0   
HS-grad       10501.0  38.974479  13.541524  17.0  37.0  90.0        10501.0   
Masters        1723.0  44.049913  11.068935  18.0  43.0  90.0         1723.0   
Preschool        51.0  42.764706  15.126914  19.0  41.0  75.0           51.0   
Prof-school     576.0  44.746528  11.962477  25.0  43.0  90.0          576.0   
Some-college   7291.0  35.756275  13.474051  17.0  34.0  90.0         7291.0   

                                                      
                   mean        std   min   50%   max  
education                                             
10th          37.052519  13.788112   1.0  40.0  99.0  
11th          33.925957  13.965416   2.0  40.0  99.0  
12th          35.780600  12.626412   6.0  40.0  99.0  
1st-4th       38.255952  12.848727   4.0  40.0  96.0  
5th-6th       38.897898  10.551727   3.0  40.0  84.0  
7th-8th       39.366873  14.201870   2.0  40.0  99.0  
9th           38.044747  11.064402   1.0  40.0  99.0  
Assoc-acdm    40.504217  12.196666   1.0  40.0  99.0  
Assoc-voc     41.610709  10.793384   1.0  40.0  99.0  
Bachelors     42.614006  11.446185   2.0  40.0  99.0  
Doctorate     46.973366  15.084447   1.0  45.0  99.0  
HS-grad       40.575374  11.333757   1.0  40.0  99.0  
Masters       43.836332  12.277801   1.0  40.0  99.0  
Preschool     36.647059  12.555196  10.0  40.0  75.0  
Prof-school   47.425347  14.806038   2.0  48.0  99.0  
Some-college  38.852284  12.761901   1.0  40.0  99.0

In [115]:
data.groupby("education")[["age", "hours-per-week"]].agg([np.mean, np.std, np.min, np.max])

age                      hours-per-week                  \
                   mean        std amin amax           mean        std amin   
education                                                                     
10th          37.429796  16.720713   17   90      37.052519  13.788112    1   
11th          32.355745  15.545485   17   90      33.925957  13.965416    2   
12th          32.000000  14.334625   17   79      35.780600  12.626412    6   
1st-4th       46.142857  15.615625   19   90      38.255952  12.848727    4   
5th-6th       42.885886  15.557285   17   84      38.897898  10.551727    3   
7th-8th       48.445820  16.092350   17   90      39.366873  14.201870    2   
9th           41.060311  15.946862   17   90      38.044747  11.064402    1   
Assoc-acdm    37.381443  11.095177   19   90      40.504217  12.196666    1   
Assoc-voc     38.553546  11.631300   19   84      41.610709  10.793384    1   
Bachelors     38.904949  11.912210   19   90      42.614006  11.446185    2   
Doctorate     47.702179  11.784716   24   80      46.973366  15.084447    1   
HS-grad       38.974479  13.541524   17   90      40.575374  11.333757    1   
Masters       44.049913  11.068935   18   90      43.836332  12.277801    1   
Preschool     42.764706  15.126914   19   75      36.647059  12.555196   10   
Prof-school   44.746528  11.962477   25   90      47.425347  14.806038    2   
Some-college  35.756275  13.474051   17   90      38.852284  12.761901    1   

                   
             amax  
education          
10th           99  
11th           99  
12th           99  
1st-4th        96  
5th-6th        84  
7th-8th        99  
9th            99  
Assoc-acdm     99  
Assoc-voc      99  
Bachelors      99  
Doctorate      99  
HS-grad        99  
Masters        99  
Preschool      75  
Prof-school    99  
Some-college   99

In [116]:
data.pivot_table(["age", "hours-per-week"], ["education"], aggfunc="mean")

,age,hours-per-week
education,,
10th,37.429796,37.052519
11th,32.355745,33.925957
12th,32.000000,35.780600
1st-4th,46.142857,38.255952
5th-6th,42.885886,38.897898
7th-8th,48.445820,39.366873
9th,41.060311,38.044747
Assoc-acdm,37.381443,40.504217
Assoc-voc,38.553546,41.610709


In [117]:
data.pivot_table(index="education", columns="sex", values="hours-per-week", aggfunc="mean")

sex,Female,Male
education,,
10th,32.111864,39.336991
11th,29.821759,36.312248
12th,31.791667,37.768166
1st-4th,31.978261,40.622951
5th-6th,36.047619,39.859438
7th-8th,36.200000,40.409465
9th,33.916667,39.651351
Assoc-acdm,37.358670,42.554180
Assoc-voc,37.830000,43.753968


In [118]:
age_plus_hours = data["age"] + data["hours-per-week"]

In [119]:
data.insert(loc=2, column='age_plus_hours', value=age_plus_hours) 
# loc - номер столбца, после которого нужно вставить данный Series. в конец - len(data.columns)

In [120]:
data.head()

,age,workclass,age_plus_hours,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,79,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,63,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,78,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,93,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,68,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [121]:
d = {"Female": 0, "Male": 1}

In [122]:
data.replace({"sex": d})

,age,workclass,age_plus_hours,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,79,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,1,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,63,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,13,United-States,<=50K
2,38,Private,78,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,1,0,0,40,United-States,<=50K
3,53,Private,93,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,1,0,0,40,United-States,<=50K
4,28,Private,68,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,0,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,65,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,0,0,0,38,United-States,<=50K
32557,40,Private,80,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,1,0,0,40,United-States,>50K
32558,58,Private,98,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,0,0,0,40,United-States,<=50K
32559,22,Private,42,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,1,0,0,20,United-States,<=50K


In [123]:
data["sex"] = data["sex"].map(d)

In [124]:
data.head()

,age,workclass,age_plus_hours,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,79,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,1,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,63,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,1,0,0,13,United-States,<=50K
2,38,Private,78,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,1,0,0,40,United-States,<=50K
3,53,Private,93,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,1,0,0,40,United-States,<=50K
4,28,Private,68,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,0,0,0,40,Cuba,<=50K


In [128]:
data["sex"] = data["sex"].apply(lambda x: "male" if x else "female")

In [129]:
data.head()

,age,workclass,age_plus_hours,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,79,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,63,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,male,0,0,13,United-States,<=50K
2,38,Private,78,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,male,0,0,40,United-States,<=50K
3,53,Private,93,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,male,0,0,40,United-States,<=50K
4,28,Private,68,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,female,0,0,40,Cuba,<=50K


In [133]:
pd.crosstab(data["sex"], data["workclass"], margins=True)

workclass,?,Federal-gov,Local-gov,Never-worked,Private,Self-emp-inc,Self-emp-not-inc,State-gov,Without-pay,All
sex,,,,,,,,,,
female,839,315,835,2,7752,135,399,489,5,10771
male,997,645,1258,5,14944,981,2142,809,9,21790
All,1836,960,2093,7,22696,1116,2541,1298,14,32561


In [134]:
############################################ homework ##################################################

In [135]:
df = pd.read_csv("../files/adult.data.csv")

In [137]:
df["sex"].value_counts()   # можно добавить normalize=[True, False]

Male      21790
Female    10771
Name: sex, dtype: int64

In [141]:
df[df["sex"] == "Female"]["age"].mean()

36.85823043357163

In [144]:
df["native-country"].value_counts(normalize=True)

United-States                 0.895857
Mexico                        0.019748
?                             0.017905
Philippines                   0.006081
Germany                       0.004207
Canada                        0.003716
Puerto-Rico                   0.003501
El-Salvador                   0.003255
India                         0.003071
Cuba                          0.002918
England                       0.002764
Jamaica                       0.002488
South                         0.002457
China                         0.002303
Italy                         0.002242
Dominican-Republic            0.002150
Vietnam                       0.002058
Guatemala                     0.001966
Japan                         0.001904
Poland                        0.001843
Columbia                      0.001812
Taiwan                        0.001566
Haiti                         0.001351
Iran                          0.001321
Portugal                      0.001136
Nicaragua                

In [169]:
df[df["salary"] == ">50K"]["age"].describe()

count    7841.000000
mean       44.249841
std        10.519028
min        19.000000
25%        36.000000
50%        44.000000
75%        51.000000
max        90.000000
Name: age, dtype: float64

In [170]:
df[df["salary"] != ">50K"]["age"].describe()

count    24720.000000
mean        36.783738
std         14.020088
min         17.000000
25%         25.000000
50%         34.000000
75%         46.000000
max         90.000000
Name: age, dtype: float64

In [151]:
df[df["salary"] == ">50K"].groupby("education").count()

,age,workclass,fnlwgt,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
education,,,,,,,,,,,,,,
10th,62,62,62,62,62,62,62,62,62,62,62,62,62,62
11th,60,60,60,60,60,60,60,60,60,60,60,60,60,60
12th,33,33,33,33,33,33,33,33,33,33,33,33,33,33
1st-4th,6,6,6,6,6,6,6,6,6,6,6,6,6,6
5th-6th,16,16,16,16,16,16,16,16,16,16,16,16,16,16
7th-8th,40,40,40,40,40,40,40,40,40,40,40,40,40,40
9th,27,27,27,27,27,27,27,27,27,27,27,27,27,27
Assoc-acdm,265,265,265,265,265,265,265,265,265,265,265,265,265,265
Assoc-voc,361,361,361,361,361,361,361,361,361,361,361,361,361,361


In [152]:
df.groupby(["race", "sex"]).agg({"age": "describe"})

age                                          \
                             count       mean        std   min   25%   50%   
race               sex                                                       
Amer-Indian-Eskimo Female    119.0  37.117647  13.114991  17.0  27.0  36.0   
                   Male      192.0  37.208333  12.049563  17.0  28.0  35.0   
Asian-Pac-Islander Female    346.0  35.089595  12.300845  17.0  25.0  33.0   
                   Male      693.0  39.073593  12.883944  18.0  29.0  37.0   
Black              Female   1555.0  37.854019  12.637197  17.0  28.0  37.0   
                   Male     1569.0  37.682600  12.882612  17.0  27.0  36.0   
Other              Female    109.0  31.678899  11.631599  17.0  23.0  29.0   
                   Male      162.0  34.654321  11.355531  17.0  26.0  32.0   
White              Female   8642.0  36.811618  14.329093  17.0  25.0  35.0   
                   Male    19174.0  39.652498  13.436029  17.0  29.0  38.0   

                                        
                             75%   max  
race               sex                  
Amer-Indian-Eskimo Female  46.00  80.0  
                   Male    45.00  82.0  
Asian-Pac-Islander Female  43.75  75.0  
                   Male    46.00  90.0  
Black              Female  46.00  90.0  
                   Male    46.00  90.0  
Other              Female  39.00  74.0  
                   Male    42.00  77.0  
White              Female  46.00  90.0  
                   Male    49.00  90.0

In [172]:
df[(df["salary"] == ">50K") & (df["sex"] == "Male")].groupby("marital-status").count()

,age,workclass,fnlwgt,education,education-num,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
marital-status,,,,,,,,,,,,,,
Divorced,284,284,284,284,284,284,284,284,284,284,284,284,284,284
Married-AF-spouse,4,4,4,4,4,4,4,4,4,4,4,4,4,4
Married-civ-spouse,5938,5938,5938,5938,5938,5938,5938,5938,5938,5938,5938,5938,5938,5938
Married-spouse-absent,23,23,23,23,23,23,23,23,23,23,23,23,23,23
Never-married,325,325,325,325,325,325,325,325,325,325,325,325,325,325
Separated,49,49,49,49,49,49,49,49,49,49,49,49,49,49
Widowed,39,39,39,39,39,39,39,39,39,39,39,39,39,39


In [159]:
df.sort_values(by="hours-per-week", ascending=False).groupby("hours-per-week").count().tail()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,native-country,salary
hours-per-week,,,,,,,,,,,,,,
95,2,2,2,2,2,2,2,2,2,2,2,2,2,2
96,5,5,5,5,5,5,5,5,5,5,5,5,5,5
97,2,2,2,2,2,2,2,2,2,2,2,2,2,2
98,11,11,11,11,11,11,11,11,11,11,11,11,11,11
99,85,85,85,85,85,85,85,85,85,85,85,85,85,85


In [166]:
df[df["hours-per-week"] == df["hours-per-week"].max()]["salary"].value_counts(normalize=True)
                          

<=50K    0.705882
>50K     0.294118
Name: salary, dtype: float64

In [173]:
df.groupby(["native-country", "salary"]).agg({"hours-per-week": "mean"}).loc["Japan"]

,hours-per-week
salary,
<=50K,41.000000
>50K,47.958333
